# Collect and merge data into single CSV

In [1]:
from pathlib import Path
import pandas as pd 
from datetime import datetime

In [2]:
# Load solar data from files and merge solar data into single dataframe

solar_data_path = Path(r"solar_data_raw").glob('*.xls')

solar_data_files = [x for x in solar_data_path if x.is_file()]

solar_data = pd.DataFrame()

for file in solar_data_files:
  
    solar_data = solar_data.append(pd.read_excel(file))

solar_data['Time'] =  pd.to_datetime(solar_data['Time'], format="%Y-%m-%d %H:%M:%S")

In [4]:
# Load historical weather_data

weather_data = pd.read_csv('weather_data_clean.csv', index_col=0)

weather_data['Time'] = pd.to_datetime(weather_data['dt_iso'], format="%Y-%m-%d %H:%M:%S")

# Drop other datetime coloumns 
weather_data = weather_data.drop(['dt','dt_iso'], axis=1)

In [ ]:
# Join solar data for weather data 

combined_data = pd.merge_asof(weather_data, solar_data.sort_values('Time'), left_on='Time', right_on='Time', direction='nearest', tolerance=pd.Timedelta("600s"))

# Drop rows with missing data

combined_data = combined_data.dropna(subset=['ePv1Day'])

combined_data = combined_data.drop(['sea_level','grnd_level', 'rain_1h', 'rain_3h', 'snow_1h', 'snow_3h'],axis=1)

# Sum energy output from each solar array to calculate total solar energy

combined_data['Solar Energy'] = combined_data['ePv1Day'] + combined_data['ePv2Day'] 

In [ ]:
# Create day averaged dataframe from combined dataframe

combined_data['date'] = combined_data['Time'].dt.date

day_df = combined_data.groupby('date').agg(
    temp_mean = ('temp','mean'), 
    temp_max = ('temp_max','max'), 
    temp_min = ('temp_min','min'), 
    pressure_mean = ('pressure','mean'),
    pressure_min = ('pressure','min'),
    pressure_max = ('pressure','max'),
    humidity_mean = ('humidity', 'mean'),
    humidity_min = ('humidity', 'min'),
    humidity_max = ('humidity', 'max'),
    wind_speed_mean = ('wind_speed', 'mean'),
    wind_speed_min = ('wind_speed', 'min'),
    wind_speed_max = ('wind_speed', 'max'),
    wind_deg_mean = ('wind_deg', 'mean'),
    wind_deg_min = ('wind_deg', 'min'),
    wind_deg_max = ('wind_deg', 'max'),
    clouds_all_mean = ('clouds_all', 'mean'),
    clouds_all_min = ('clouds_all', 'min'),
    clouds_all_max = ('clouds_all', 'max'),
    weather_id_mode = ('weather_id', lambda x: x.value_counts().index[0]),
    total_solar = ('Solar Energy','max')
    ).reset_index()

day_df.date = pd.to_datetime(day_df['date'], format="%Y-%m-%d")

day_df['date'] = pd.to_datetime(day_df['date'])
day_df['Day'] = day_df['date'].apply(lambda x: (x.to_pydatetime().date() - datetime(x.to_pydatetime().year, 1,1 ).date()))
day_df['Day'] = day_df['Day'].dt.days

day_df.to_csv('day_averaged.csv', index=False)